$$ \hat{G}\hat{K}\hat{G} = \sum_{i=0}^{N-1}\sum_{\sigma}\prod_{j \neq i,i+1}e^{-2g n_{j\uparrow}n_{j\downarrow}} e^{-g n_{i\sigma+1}}e^{-g n_{i+1\sigma+1}}  \frac{k}{2}\left(X_{i\sigma}X_{i+1\sigma}+Y_{i+1\sigma}Y_{i\sigma}\right) \rightarrow \sum_{i=0}^{N-1}\sum_{\sigma}\prod_{j \neq 0,1}e^{-2g n_{j\uparrow}n_{j\downarrow}} e^{-g n_{0\sigma+1}}e^{-g n_{1\sigma+1}}  \frac{k}{2}\left(Z_0 - Z_1\right)$$

\begin{equation}
\begin{split}
G_2 K G_2 =\sum_{i\sigma} \left(\prod_{j\neq i-1,i,i+1} e^{-2g n_j n_{j+1}}\right) e^{-g n_{i-1}(1+2n_{i\sigma+1})}e^{-g (n_{i+1\sigma+1}+n_{i\sigma+1})^2}e^{-g (1+2n_{i+1\sigma+1})n_{i+2}} \left( X_{i\sigma}X_{i+1\sigma} + Y_{i+1\sigma}Y_{i\sigma} \right) \rightarrow \sum_{i\sigma} \left(\prod_{j\neq N,0,1} e^{-2g n_j n_{j+1}}\right) e^{-g n_{N}(1+2n_{0\sigma+1})}e^{-g (n_{1\sigma+1}+n_{0\sigma+1})^2}e^{-g (1+2n_{1\sigma+1})n_{2}} \left( Z_0 - Z_1 \right) 
\end{split}
\end{equation}

In [5]:
%%writefile Define_NN_Analyzers.py

import numpy as np

#Generates the set of indecies after the Pauli X and Y terms have been taken to sites 0 and 1
#fset holds the indeces 0 or 1 which do not have X or Y in the original Pauli string
#hset holds the indeces > 1 which have X or Y in the original Pauli sting
#The new pauli string will have indces in fset swapped with those in hset
def find_sset(pauli):
    sset = [i for i in range(len(pauli))] #Swapped set
    fset = []
    hset = []
    N=len(pauli)
    for i in range(2):
        if pauli[i] == 'Z' or pauli[i] == 'I':
            fset.append(i)
    for i in range(2,len(pauli)):
        if pauli[i] == 'X' or pauli[i] == 'Y':
                hset.append(i)
    for i in range(len(fset)):
        sset[hset[i]] = fset[i]
        sset[fset[i]] = hset[i]
    return sset

#Finds the total number of particles in a state
#This is used for post selection
def n_total(state):
    n_total = 0
    for i in range(len(state)):
        n_total += int(state[i])
    return n_total

# Returns the expectation value of <G2(g)G2(g)> using the result measured in the z-basis 
def analyze_GG2(g,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr
    return gg

# Returns the expectation value of <G2(g)D(d)G2(g)> using the result measured in the z-basis 
def analyze_GDG2(g,d,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            d_sr = 0
            for i in range(len(su)):
                nu = int(su[i])
                nd = int(sd[i])
                d_sr = d_sr + d*nu*nd
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*d_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*d_sr
    return gg

# Returns the expectation value of G2(g)M(m)G2(g) using the result measured in the z-basis 
def analyze_GMG2(g,m,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            m_sr = 0
            for i in range(len(su)):
                nu = int(su[i])
                nd = int(sd[i])
                m_sr = m_sr + m*nu + m*nd
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*m_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*m_sr
    return gg

# This returns the results for <Gb> where G2 P G2 = P Gb 
# This is not used in the final construction but can be useful for debugging
def analyze_GG2b(g,nbrs,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            gg_sr = 1
            for pair in nbrs:
                ui = N-1-pair[0]
                di = N-1-sset[pair[0]]
                uj = N-1-pair[1]
                dj = N-1-sset[pair[1]]
                nui = int(su[ui])
                ndi = int(sd[di])
                ni = nui + ndi
                nuj = int(su[uj])
                ndj = int(sd[dj])
                nj = nuj + ndj
                if di == 0 and dj == 1:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2) 
                elif dj == 0 and di == 1:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2)
                elif di == 0 or di == 1:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*nj)
                elif dj == 0 or dj == 1:
                    gg_sr = gg_sr*np.exp(-g*ni*(1+2*nuj))
                else:
                    gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr
    return gg

# Returns the expectation value of G2(g)P G2(g) where P is a pauli string using the results for P
def analyze_GPG2(g,nbrs,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            gg_sr = 1
            z0 = 1
            z1 = 1
            if sd[N-1] == '1':
                z0 = -1
            if sd[N-2] == '1':
                z1 = -1
            k_sr = z0 - z1
            N  = len(su)
            for pair in nbrs:
                ui = N-1-pair[0]
                di = N-1-sset[pair[0]]
                uj = N-1-pair[1]
                dj = N-1-sset[pair[1]]
                nui = int(su[ui])
                ndi = int(sd[di])
                ni = nui + ndi
                nuj = int(su[uj])
                ndj = int(sd[dj])
                nj = nuj + ndj
                if di == N-1 and dj == N-2:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2) 
                elif dj == N-1 and di == N-2:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2)
                elif di == N-1 or di == N-2:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*nj)
                elif dj == N-1 or dj == N-2:
                    gg_sr = gg_sr*np.exp(-g*ni*(1+2*nuj))
                else:
                    gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*k_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*k_sr 
    return gg

# Returns the expection value of <G(g)K(k)G(g)> using the results for each pauli string 
def analyze_GKG2(g,k,nbrs,paulis,results, post_select = True):
    out = 0
    for p in range(1,len(paulis),2):
        pauli = paulis[p]
        out += k/2*analyze_GPG2(g,nbrs,pauli,paulis,results, post_select = post_select) #for spin up
        out += k/2*analyze_GPG2(g,nbrs,pauli,paulis,results, post_select = post_select) #for spin down
    return out

Writing Define_NN_Analyzers.py


# Tests

In [2]:
from Define_System import system
from Define_Slater_Circuit import slater_circ
from Define_Measurment_Circuits import create_circs
from Define_Quantum_Device import Quantum_Device
from Define_Analyzers import analyze_energy, post_select
from Define_Lancozos import Lancozos
systemQ = system('triangle',4) 
#systemQ = system('square',4) 
Fd = systemQ.Fld()
paulis = systemQ.pauli_strings()
circs = create_circs(Fd,paulis)
QD = Quantum_Device(backend = 'ibm_auckland', layout = [0,1,2,3,5,8,11,14])

results = QD.get_results(circs ,method = "matrix")
results = QD.chop_results(results)

In [3]:
from Define_Paulis import bkt,Mdot,X,Y,Z,I,c,cd

g = 0.7
k = 1.4
g2 = 0.6
d = 0.4
m = 1.3
psi_spin = systemQ.psi_spin([2,3])

In [4]:
nbrs = systemQ.neighbors()


G2 = systemQ.G2(g2)
D = systemQ.D(d)
M = systemQ.M(m)
K = systemQ.K(k)
i = 0; j = 1;
pn = 1
GPG2 = Mdot([psi_spin,G2,cd(i,8),c(j,8),G2,psi_spin]) + Mdot([psi_spin,G2,cd(j,8),c(i,8),G2,psi_spin])

print( Mdot([psi_spin,G2,G2,psi_spin]) - analyze_GG2(g2,nbrs,results[0]) )
print( Mdot([psi_spin,G2,D,G2,psi_spin]) - analyze_GDG2(g2,d,nbrs,results[0]) )
print( Mdot([psi_spin,G2,M,G2,psi_spin]) - analyze_GMG2(g2,m,nbrs,results[0]) )
print( GPG2 - 1/2*analyze_GPG2(g2,nbrs,paulis[pn],paulis,results) )
print( Mdot([psi_spin,G2,K,G2,psi_spin]) - analyze_GKG2(g2,k,nbrs,paulis,results) )

(1.3552527156068805e-20+0j)
(6.776263578034403e-21+0j)
(2.710505431213761e-20+0j)
(-1.6940658945086007e-21+0j)
0j
